# Nearest Neighbor Distance Distribution

##### Load some packages and previous functions

In [1]:
#The following packages are required to read and write files, as well as generate linear algebra calculations
using DelimitedFiles
using LinearAlgebra
using Plots

#The following files with Julia functions are required
include("Operaciones_Basicas.jl")
include("Red_Dual.jl")
include("Separacion_Arreglos_X_Y.jl")
include("Generador_Vecindades_Pto_Arbitrario.jl")
include("voronoi.jl")
include("Algoritmo_Voronoi.jl")
include("Algoritmo_Cluster_Principal.jl")
include("Diccionario_Indices_Centroides_Voronoi.jl")
include("Algoritmo_Distancia_Primer_Vecino.jl")

third_Neighbor_Distance (generic function with 1 method)

#### We generate the parameters that determine the type of quasiperiodic arrangement that we are going to build

In [2]:
NSides = 5; #Rotational symmetry of the quasiperiodic lattice
Star_Vectors = [[BigFloat(1),0]]; #Arrangement that will contain the star vectors
for i in 1:(NSides-1)
    push!(Star_Vectors, [cos((2*i)*pi/NSides), sin((2*i)*pi/NSides)]); #Vertices of the polygon with "NSides" sides
end
Alphas_Array = fill(0.2, NSides); #Array with the alpha constants of the GDM
Average_Distance_Stripes = fill(NSides/2, NSides); #Array with the average spacing between stripes

#Information from the patch
Error_Margin = 5; #Margin of error in the integers "n" associated with the star vectors of the GDM
SL = 1e6; #Size of the half side of the square centered on the (0,0) where a starting position is sought.
Bounded_Area = 1.2; #The value for the area of the polygons that will be a discriminator value in the areas algorithm
Reduction_Factor = 0.7; #Factor to generate a "safe radius"

#### Let's iterate the function to get the distribution of the first neighbor distance

In [3]:
Number_Vertices = 100000; #Total number of vertices to analyze
Complete_Array_FND = []; #Array with all the Nearest Neighbor Distances of all the patches
i=1; #Counter to know the actual iteration

while length(Complete_Array_FND) < Number_Vertices
    if mod(i,10) == 0
        open("Estado_TND_N_$(NSides).txt", "w") do file
            write(file, "Estamos calculando el parche $(i); se han analizado $(length(Complete_Array_FND)) vértices.")
        end
    end
    Tercer_Vecino_Array = third_Neighbor_Distance(Error_Margin, SL, Bounded_Area, Reduction_Factor, Average_Distance_Stripes, Star_Vectors, Alphas_Array);
    Complete_Array_FND = vcat(Complete_Array_FND, Tercer_Vecino_Array)
    i+=1;
end

In [4]:
writedlm("Nearest_Neighbor_Distance_N_$(NSides)_H_3.csv", Complete_Array_FND, ',')

# Algorithm to read csv and get the tuples

In [5]:
AA = readdlm("Nearest_Neighbor_Distance_N_$(NSides)_H_3.csv");
Tuples_Array = [];

for e in AA
    B = split(e, ",")
    x = (parse(Float64, B[1]), parse(Float64, B[2]), parse(Float64, B[3]))
    push!(Tuples_Array, x)
end